## Exploring the MPC's Isolated Tracklet File
#### Matt Holman, Matt Payne, Paul Blankley, Ryan Janssen


1 December 2017

This notebook is intended to be a consolidation of the material that is in ITF_mjh_20171201.ipynb and ITF_mjh.ipynb

The plan is to:

    * Remove unused routines.
    * Select a final, working approach.
    * Set up training/test routines.
    * Set up routines to run on the full ITF.


### The NOVAS package

First, get the USNO's python NOVAS package.  We'll need that.

http://aa.usno.navy.mil/software/novas/novas_py/novaspy_intro.php

Just type 

pip install novas

pip install novas_de405

Here's the reference:

Barron, E. G., Kaplan, G. H., Bangert, J., Bartlett, J. L., Puatua, W., Harris, W., & Barrett, P. (2011) “Naval Observatory Vector Astrometry Software (NOVAS) Version 3.1, Introducing a Python Edition,” Bull. AAS, 43, 2011.

### The kepcart library

You will need to make sure you have a copy of the kepcart library.  There is a copy of it on the MPC bitbucket site, with some instructions.

In [1]:
%matplotlib inline
import numpy as np
import scipy.interpolate
import matplotlib.gridspec as gridspec
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 200)
pd.set_option('display.notebook_repr_html', True)
import math
import kepcart as kc
import healpy as hp
import collections
import astropy
from collections import defaultdict
from collections import Counter
import MPC_library
import scipy.spatial
import pickle
from operator import add

Observatories = MPC_library.Observatories

ObservatoryXYZ = Observatories.ObservatoryXYZ

### Reading the MPC observation files

Dealing with files line by line in python is not fast.  

The itf.txt, NumObs.txt, and UnnObs.txt files have a mix of 1-line and 2-line formats.  


In [2]:
# This routine checks the 80-character input line to see if it contains a special character (S, R, or V) that indicates a 2-line 
# record.
def is_two_line(line):
    note2 = line[14]
    return note2=='S' or note2=='R' or note2=='V'

In [3]:
# This routine opens and reads filename, separating the records into those in the 1-line and 2-line formats.
# The 2-line format lines are merged into single 160-character records for processing line-by-line.
def split_MPC_file(filename):
    filename_1_line = filename.rstrip('.txt')+"_1_line.txt"
    filename_2_line = filename.rstrip('.txt')+"_2_line.txt"
    with open(filename_1_line, 'w') as f1_out, open(filename_2_line, 'w') as f2_out:
        line1=None
        with open(filename, 'r') as f:
            for line in f:
                if is_two_line(line):
                    line1=line
                    continue
                if line1 != None:
                    merged_lines = line1.rstrip('\n') + line
                    f2_out.write(merged_lines)
                    line1 = None
                else:
                    f1_out.write(line)
                    line1 = None


Now split the files for the three main MPC observation files.  (Need to convert the section below to code from markdown, if you haven't run it already.)

Removed the 2-line observations, for now.

split_MPC_file('itf_new.txt')

Below are routines that read the files after they have been split into their respective formats.  

In [4]:
def readMPC_1_line(filename='NumObs_1_line.txt', nrows=1000000):
    colspecs = [(0, 5), (5, 12), (12, 13), (13, 14), (14, 15), (15, 32), (32, 44), (44, 56), (65, 71), (77, 80)]
    colnames = ['objName', 'provDesig', 'disAst', 'note1', 'note2', 'dateObs', 'RA', 'Dec', 'MagFilter', 'obsCode']
    df = pd.read_fwf(filename, colspecs=colspecs, names=colnames, header=None, nrows=nrows)
    return df

In [5]:
def convertObs80(line):
    objName   = line[0:5]
    provDesig = line[5:12]
    disAst    = line[12:13]
    note1     = line[13:14]
    note2     = line[14:15]
    dateObs   = line[15:32]
    RA        = line[32:44]
    Dec       = line[44:56]
    mag       = line[65:70]
    filt      = line[70:71]
    obsCode   = line[77:80]
    return objName, provDesig, disAst, note1, note2, dateObs, RA, Dec, mag, filt, obsCode


In [6]:
def splitMagFilter(magFilter):
    pieces = magFilter.split()
    if len(pieces)==0:
        return None, None
    elif len(pieces)==1:
         return pieces[0], None
    else:
        return pieces[0], pieces[1]

        

Here I am generating a version of the MPC data that includes just the few string-like fields at the start of each line, followed by jd_tdb, the unit vector to the observed target, and the heliocentric position vector of the observatory at that time.

CS0051 had a typo on the magnitude, a missing '.' in the number.  I corrected that manually.

We need to preserve tracklets across the time and space boundaries.  That means that the division into time slices and spatial regions should be based on the first member of a tracklet only.  That way the full tracklet is accepted or not, rather than pieces occasionally.

The routine below does that, returning a dictionary with trackletIDs as keys and a list of the corresponding observation lines as values.  It also returns a dictionary that gives the starting jd_tdb for each tracklet, as well as a list of tracklets in time-sorted order.

In [7]:
def get_sorted_tracklets(itf_filename):
    tracklets = defaultdict(list)
    tracklets_jd_dict = {}
    with open(itf_filename) as infile:
        for line in infile:
            if not line.startswith('#'):
                desig = line[0:12]
                jd_tdb = float(line[43:57])
                if desig not in tracklets_jd_dict:
                    # Got a new tracklet
                    tracklets_jd_dict[desig] = jd_tdb
                tracklets[desig].append(line)
    sortedTrackletKeys = sorted(tracklets.keys(), key=lambda k: tracklets_jd_dict[k]) 
    return tracklets, tracklets_jd_dict, sortedTrackletKeys

Gather up this supporting information for the ITF first.

In [ ]:
tracklets, tracklets_jd_dict, sortedTracklets = get_sorted_tracklets('data/itf_new_1_line.mpc')

for k in sortedTracklets[:10]:
    print(k, tracklets_jd_dict[k])    

len(sortedTracklets)

Now gather up the corresponding supporting information for the UnnObs (unnumbered observations) training set that Matt Payne prepared.

In [ ]:
UnnObs_tracklets, UnnObs_tracklets_jd_dict, UnnObs_sortedTracklets = get_sorted_tracklets('data/UnnObs_Training_1_line_A.mpc')

for k in UnnObs_sortedTracklets[:10]:
    print(k, UnnObs_tracklets_jd_dict[k])    

len(UnnObs_sortedTracklets)

We want to separate the data into time windows that are centered on each new moon, so here's a function that takes an integer and returns a corresponding julian date of new moon, rounded to midnight UTC.  Note that this function is approximate but good enough for our purposes.

In [ ]:
# Returns the jd of new moon, to the nearest half day
def lunation_center(n, tref=2457722.0125, p=29.53055):
    t = tref + p*n
    tp = np.floor(t) + 0.5
    return tp

Sweep through the tracklets once, outputting them into a sequence of overlapping time ranges that can be processed separately.



In [ ]:
def separate_time_windows(tracklets, sortedTracklets, tracklets_jd_dict, file_stem='data/itf_new_1_line.mpc', n_begin=-825, n_end=14, dt=15.):
    t_center = lunation_center(n_begin)
    files = {}

    header='#trackletID yr   mn dy      obsCode mag filter  jd_tdb       x_target     y_target     z_target      x_obs       y_obs        z_obs     '

    for desig in sortedTracklets:
        jd_tdb = tracklets_jd_dict[desig]
        while(jd_tdb>t_center+dt):
            if n_begin in files:
                files[n_begin].close()
            n_begin +=1
            t_center = lunation_center(n_begin)
        for n in range(n_begin, n_end):
            if jd_tdb<lunation_center(n)-dt:
                break
            if n not in files:
                outfile = file_stem.rstrip('.mpc')+'_'+str(lunation_center(n))+'_pm'+str(dt)+'.mpc'
                files[n] = open(outfile, 'w')
                files[n].write(header+'\n')
            for line in tracklets[desig]:
                files[n].write(line)


Separate the ITF data and UnnObs training files into overlapping +/- 15 day windows (the default).

The +/-45 day range we were exploring earlier is a little too broad for this model, at present.

This only needs to be done once.

Now get ready to transform the data assuming a heliocentric observer.  The point with this is to be able to figure out which tracklets would be observable, so that we can bring those data together for processing.  The function below will get called repeatedly.

In [ ]:
# This returns the topocentric distances and new heliocentric
# position vectors to the target, given the assumed distance
# r and the position vector of the observatory re.
def adjust_position(r, rho_target, re):
    rho_x, rho_y, rho_z = rho_target
    xe, ye, ze = re
    Robs = np.sqrt(xe * xe + ye * ye + ze * ze)
    cos_phi = -(rho_x * xe + rho_y * ye + rho_z * ze) / Robs
    phi = np.arccos(cos_phi)
    sin_phi = np.sin(phi)

    xx2 = r*r - Robs*sin_phi * Robs*sin_phi
    
    if xx2 < 0:
        None, None

    xx = np.sqrt(xx2)
    yy = Robs * cos_phi
    
    rho_p = yy + xx

    # This could be done with numpy arrays
    x_p = xe + rho_p*rho_x
    y_p = ye + rho_p*rho_y
    z_p = ze + rho_p*rho_z
    
    rho_m = yy - xx
    
    # This could be done with numpy arrays    
    x_m = xe + rho_m*rho_x
    y_m = ye + rho_m*rho_y
    z_m = ze + rho_m*rho_z
        
    return (rho_p, (x_p, y_p, z_p)), (rho_m, (x_m, y_m, z_m))


Now do the heliocentric transformation, but (as noted above) it is just for the purpose of figuring out where the tracklets would appear in the sky from the sun so that we can select chunks of data.

In [ ]:
# Does the transformations on the data using the date of the n-th new
# moon as the reference time.
#
# It is reading and processing the entire *.mpc file.
#
# This does the heliocentric tranformation for the assumed radius function,
# r_func.
#
# It then does light-time correction.
#
# And it appends a healpix number on each line in order to be able to quickly 
# select data from a given region of sky.
#
# This generates a file called *.trans, and it incorporates
# the distance assumed in the file name.
#
def transform_positions(n, r_func, file_stem='data/itf_new_1_line.mpc', dt=45., nside=8):
    infilename = file_stem.rstrip('.mpc')+'_'+str(lunation_center(n))+'_pm'+str(dt)+'.mpc'
    try:
      open(infilename, 'r')
    except IOError:
      return 0    
    t_ref = lunation_center(n)
    r_ref = r_func(t_ref)
    r_name = "_r%.1lf" % (r_ref)
    outfilename = file_stem.rstrip('.mpc')+'_'+str(lunation_center(n))+'_pm'+str(dt)+r_name+'.trans'

    #rot_mat = MPC_library.rotate_matrix(-MPC_library.Constants.ecl)
    
    with open(infilename, 'r') as infile, open(outfilename, 'w') as outfile:
        for line in infile:
            if line.startswith('#'): 
                header = line.rstrip()
                outfile.write(header + '          dt         x_cor       y_cor        z_cor       pix \n')
            else:
                lineID = line[:43]

                jd_tdb = float(line[43:57])

                x_target, y_target, z_target = line[57:97].split()
                r_target = np.array([float(x_target), float(y_target), float(z_target)])

                x_obs, y_obs, z_obs = line[97:135].split()            
                r_obs = np.array([float(x_obs), float(y_obs), float(z_obs)])
                
                # This should be a function from here
                # Adjust positions
                dt = 0.0
                r_prev = r_func(jd_tdb-dt)
                rho_r_p, rho_r_m = adjust_position(r_prev, r_target, r_obs)
                dt = rho_r_p[0]/MPC_library.Constants.speed_of_light
                
                # Do light-time iterations.
                # Probably don't need to do this at this point, because it is
                # being re-done in a later step.
                i=0
                while(np.abs(r_func(jd_tdb-dt)-r_prev)>1e-8):
                    rho_r_p, rho_r_m = adjust_position(r_prev, r_target, r_obs)
                    dt = rho_r_p[0]/MPC_library.Constants.speed_of_light
                    r_prev = r_func(jd_tdb-dt)
                    i += 1
    
                # to here
                xp, yp, zp = rho_r_p[1]
                
                # Calculate HEALPix index
                pix = hp.vec2pix(nside, xp, yp, zp, nest=True)
            
                outstring = line.rstrip() + " %13.6lf %12.7lf %12.7lf %12.7lf %5d\n"% \
                      (dt, xp, yp, zp, pix)

                outfile.write(outstring)

        

Do the transformation for the ITF with a +/- 15 day window.

This only needs to be done once (per distance class).  And after this step the *.mpc files are not needed.

Do the transformation for the UnnObs file for a +/- 15 day window.

In [ ]:
# Check again
# This routine returns the 3-D rotation matrix for the 
# given reference vector.
def xyz_to_proj_matrix(r_ref):
    x_ref, y_ref, z_ref = r_ref
    r = np.sqrt(x_ref*x_ref + y_ref*y_ref + z_ref*z_ref)
    lon0 = np.arctan2(y_ref, x_ref)
    lat0 = np.arcsin(z_ref/r)
    slon0 = np.sin(lon0)
    clon0 = np.cos(lon0)
    slat0 = np.sin(lat0)
    clat0 = np.cos(lat0)

    mat = np.array([[-slon0, clon0, 0], 
                    [-clon0*slat0, -slon0*slat0, clat0], 
                    [clon0*clat0, slon0*clat0, slat0 ]])
    
    return mat


In [ ]:
def fit_tracklet(t_ref, g, gdot, v, GM=MPC_library.Constants.GMsun):
    # Here's a version that incorporates radial gravitational
    # acceleration
    
    t_emit = [(obs[0]-obs[1]-t_ref) for obs in v]
    acc_z = -GM*g*g
    fac =[(1.0 + gdot*t + 0.5*g*acc_z*t*t - g*obs[7]) for obs, t in zip(v, t_emit)]
                
    A = np.vstack([t_emit, np.ones(len(t_emit))]).T 
    
    x = [obs[2]*f + obs[5]*g for obs, f in zip(v, fac)]                 
    mx, cx = np.linalg.lstsq(A, x)[0]
            
    y = [obs[3]*f + obs[6]*g for obs, f in zip(v, fac)]                 
    my, cy = np.linalg.lstsq(A, y)[0]
    
    return (cx, mx, cy, my, t_emit[0])

In [ ]:
def fit_tracklet_grav(t_ref, g, gdot, v, GM=MPC_library.Constants.GMsun, eps2=1e-16):
    # Here's a more sophisticated version
        
    t_emit = [(obs[0]-obs[1]-t_ref) for obs in v]
        
    # We will approximate g_x(t), g_y(t), and g_z(t)
    # using a Taylor expansion.  
    # The first two terms are zero by design.  
    #
    # Given alpha, beta, gamma,
    # we would have r_0^2 = (alpha^2 + beta^2 + 1)*z_0^2
    # r^2 = (alpha^2 + beta^2 + 1)/gamma^2 ~ 1/gamma^2
    # g_x(t) ~ -0.5*GM*x_0*t^2/r^3 + 1/6*jerk_x*t*t*t
    # g_y(t) ~ -0.5*GM*y_0*t^2/r^3 + 1/6*jerk_y*t*t*t
    # g_z(t) ~ -0.5*GM*z_0*t^2/r^3 + 1/6*jerk_z*t*t*t
    #
    # We do not have alpha and beta initially,
    # but we assert gamma.
    # 
    # We set alpha=beta=0 initially, least squares solve
    # the tracklets and obtain alpha, alpha-dot, beta, 
    # and beta-dot.
    # 
    # Then we use those values to estimate g_x,
    # g_y, and g_z for the next iteration.
    #
    # The process converges when alpha, alpha-dot,
    # beta, beta-dot do not change significantly.
    #
    # We could also do this same process with a 
    # Kepler-stepper or a full n-body integration.
    
    alpha = beta = 0.0
    alpha_dot = beta_dot = 0.0
    cx, cy = 1.0, 1.0
    mx, my = 0.0, 0.0
        
    while(((cx-alpha)*(cx-alpha) + (cy-beta)*(cy-beta))>eps2):
            
        alpha, beta = cx, cy
        alpha_dot, beta_dot = mx, my
            
        r2 = (alpha*alpha + beta*beta + 1.0)/(g*g)
        r3 = r2*np.sqrt(r2)
        r5 = r2*r3

        x0 = alpha/g
        y0 = beta/g
        z0 = 1.0/g
            
        vx0 = alpha_dot/g
        vy0 = beta_dot/g
        vz0 = gdot/g
            
        # An alternative to the Taylor expansion is to
        # to kepler step from
        # x0, y0, z0 and vx0, vy0, vz0 at time 0
        # to those at the times of each observation 
        # in the tracklet.  With that there will be no 
        # issue of convergence.
        # Then simply subtract off the linear motion
        # to give the gravitational perturbation.
            
        rrdot = x0*vx0 + y0*vy0 + z0*vz0
        
        acc_x = -GM*x0/r3
        acc_y = -GM*y0/r3
        acc_z = -GM*z0/r3
            
        jerk_x = -GM/r5*(r2*vx0 - 3.0*rrdot*x0)
        jerk_y = -GM/r5*(r2*vy0 - 3.0*rrdot*y0)
        jerk_z = -GM/r5*(r2*vz0 - 3.0*rrdot*z0)

        fac =[(1.0 + gdot*t + 0.5*g*acc_z*t*t + (1./6.0)*g*jerk_z*t*t*t - g*obs[7]) for obs, t in zip(v, t_emit)]
                
        A = np.vstack([t_emit, np.ones(len(t_emit))]).T 

        x = [obs[2]*f + obs[5]*g - 0.5*g*acc_x*t*t - (1./6.0)*g*jerk_x*t*t*t for obs, f, t in zip(v, fac, t_emit)]                 
        mx, cx = np.linalg.lstsq(A, x)[0]
            
        y = [obs[3]*f + obs[6]*g - 0.5*g*acc_y*t*t - (1./6.0)*g*jerk_y*t*t*t for obs, f, t in zip(v, fac, t_emit)]                 
        my, cy = np.linalg.lstsq(A, y)[0]
        
        return (cx, mx, cy, my, t_emit[0])

In [ ]:
# This is the one to use.  This routine will be used repeatedly.
#
# Trying a slightly different approach.
# The set of lines that are being passed in have
# been selected to be in the same region of sky
# for an assumed distance.  We are going to re-transform
# those assuming a fixed z (or gamma) value with respect
# to the sun and the reference direction, rather than a 
# fixed r, at the reference time
# 
# Rotate observatory positions to projection coordinates, 
# and recalculate simple z-based light-time correction.
#
# Rotate the observations to projection coordinates,
# but they will be theta_x, theta_y only
#
# Fit the simple abg model, for fixed gamma and
# possibly gamma_dot.
#
def select_positions_z(t_ref, g, gdot, vec, lines, outfilename, fit_tracket_func=fit_tracklet):
    
    #GM = MPC_library.Constants.GMsun

    # This rotation is taking things from equatorial to ecliptic
    rot_mat = MPC_library.rotate_matrix(MPC_library.Constants.ecl)

    results_dict = defaultdict(list)
    
    # vec is the reference direction in equatorial coordinates
    # so we rotate to ecliptic, because we want to.
    vec = np.array(vec)
    ecl_vec = np.dot(rot_mat, vec)
    vec = ecl_vec    
    vec = vec/np.linalg.norm(vec)
    # mat is a rotation matrix that converts from ecliptic
    # vectors to the projection coordinate system.
    # The projection coordinate system has z outward,
    # x parallel to increasing ecliptic longitude, and
    # y northward, making a right-handed system.
    mat = xyz_to_proj_matrix(vec)
    
    # Loop over all the lines from a *.trans file.
    for line in lines:
        if line.startswith('#'): 
            # Concatenate all header lines?
            header = line.rstrip()
        else:
            lineID = line[:43]
            trackletID = line[0:12]
                
            jd_tdb = float(line[43:57])
            dtp = float(line[139:150])
            
            # Get unit vector to target
            x_target, y_target, z_target = line[58:98].split()
            r_target = np.array([float(x_target), float(y_target), float(z_target)])
            
            # Rotate to ecliptic coordinates
            r_target_ec = np.dot(rot_mat, r_target)
            
            # Rotate to projection coordinates
            theta_x, theta_y, theta_z = np.dot(mat, r_target_ec)

            # Ignore theta_z after this; it should be very nearly 1.
            
            # Get observatory position, ultimately in projection coordinates.
            x_obs, y_obs, z_obs = line[98:138].split()
            r_obs = np.array([float(x_obs), float(y_obs), float(z_obs)])
            
            # Rotate to ecliptic coordinates
            r_obs_ec = np.dot(rot_mat, r_obs)
            
            # Rotate to projection coordinates            
            xe, ye, ze = np.dot(mat, r_obs_ec)
            
            # This is the light travel time
            dlt = ze/MPC_library.Constants.speed_of_light
                
            # Append the resulting data to a dictionary keye do trackletID.
            results_dict[trackletID].append((jd_tdb, dlt, theta_x, theta_y, theta_z, xe, ye, ze))

    # Now that we have the observations for each tracklet gathered together,
    # we iterate through the tracklets, doing a fit for each one.
    results = []
    for k, v in results_dict.items():  
        
        cx, mx, cy, my, t0 = fit_tracklet_func(t_ref, g, gdot, v)
        outstring = "%12s %16.9lf %16.9lf %16.9lf %16.9lf %16.9lf\n" % (k, cx, mx, cy, my, t0)
        
        '''
            
        # Here's a version that incorporates radial gravitational
        # acceleration

        t_emit = [(obs[0]-obs[1]-t_ref) for obs in v]
        acc_z = -GM*g*g
        fac =[(1.0 + gdot*t + 0.5*g*acc_z*t*t - g*obs[7]) for obs, t in zip(v, t_emit)]
                
        A = np.vstack([t_emit, np.ones(len(t_emit))]).T 

        # Can I put a simple acc_x term in here?
        x = [obs[2]*f + obs[5]*g for obs, f in zip(v, fac)]                 
        mx, cx = np.linalg.lstsq(A, x)[0]
            
        y = [obs[3]*f + obs[6]*g for obs, f in zip(v, fac)]                 
        my, cy = np.linalg.lstsq(A, y)[0]

        outstring = "%12s %16.9lf %16.9lf %16.9lf %16.9lf %16.9lf\n" % (k, cx, mx, cy, my, t_emit[0])
        '''
        
        results.append(outstring)

    if len(results)>0:
        with open(outfilename, 'w') as outfile:
            outstring = '#  g = %lf\n' % (g)
            outfile.write(outstring)
            outstring = '#  gdot = %lf\n' % (gdot)
            outfile.write(outstring)        
            outstring = '#  vec= %lf, %lf, %lf\n' % (vec[0], vec[1], vec[2])
            outfile.write(outstring)
            outstring = '#  desig              alpha         alpha_dot       beta             beta_dot         dt \n'
            outfile.write(outstring)
            for outstring in results:
                outfile.write(outstring)

In [ ]:
# Here I am doing the same thing as the previous routine, but without files.
#
# It takes a reference time (t_ref), a set of z, zdot pairs (z_zdot_pairs), 
# a reference direction vector (vec), and a set of observation lines that 
# have been selected for a region of sky and time slice (lines)
#
# It returns a dictionary of results that have z, zdot pairs as keys and
# sets of fitted tracklets as results.  Each result has the form:
# 
# trackletID alpha alpha_dot beta beta_dot t_emit,
# where t_emit is the light time-corrected time relative to the reference
# time.  The coordinates are now in tangent plane projection.
#
def cluster_positions_z(t_ref, g_gdot_pairs, vec, lines, fit_tracklet_func=fit_tracklet):
        
    #GM = MPC_library.Constants.GMsun

    rot_mat = MPC_library.rotate_matrix(MPC_library.Constants.ecl)

    results_dict = defaultdict(list)
    
    vec = np.array(vec)
    ecl_vec = np.dot(rot_mat, vec)
    vec = ecl_vec    
    vec = vec/np.linalg.norm(vec)
    mat = xyz_to_proj_matrix(vec)
    
    for line in lines:
        if line.startswith('#'): 
            header = line.rstrip()
        else:
            lineID = line[:43]
            trackletID = line[0:12]
                
            jd_tdb = float(line[43:57])
            dtp = float(line[139:150])
            
            # Get unit vector to target
            x_target, y_target, z_target = line[58:98].split()
            r_target = np.array([float(x_target), float(y_target), float(z_target)])
            
            # Rotate to ecliptic coordinates
            r_target_ec = np.dot(rot_mat, r_target)
            
            # Rotate to projection coordinates
            theta_x, theta_y, theta_z = np.dot(mat, r_target_ec)

            # Ignore theta_z after this; it should be very nearly 1.
            
            # Get observatory position
            x_obs, y_obs, z_obs = line[98:138].split()
            r_obs = np.array([float(x_obs), float(y_obs), float(z_obs)])
            
            # Rotate to ecliptic coordinates
            r_obs_ec = np.dot(rot_mat, r_obs)
            
            # Rotate to projection coordinates            
            xe, ye, ze = np.dot(mat, r_obs_ec)
            
            dlt = ze/MPC_library.Constants.speed_of_light
                
            results_dict[trackletID].append((jd_tdb, dlt, theta_x, theta_y, theta_z, xe, ye, ze))
            
    # All the work done above is independent of the z0 and zdot0 values

    master_results = {}
    for g_gdot in g_gdot_pairs:
        g, gdot = g_gdot

        results = []
        for k, v in results_dict.items():  
            
            cx, mx, cy, my, t0 = fit_tracklet_func(t_ref, g, gdot, v)
            result = (k, cx, mx, cy, my, t0)
            results.append(result)

        master_results[g_gdot] = results
    
    return master_results


Now do the above process over all the sky regions, for a given time slice. 

In [ ]:
# Can probably pass in the clustering function, to reduce code duplication
def cluster_sky_regions(g_gdot_pairs, pixels, t_ref, infilename, nside=8, angDeg=7.5, cluster_func=cluster_positions_z):
    
    # This bit from here
    hp_dict = defaultdict(list)
    with open(infilename) as file:
        for line in file:
            if line.startswith('#'):
                continue
            pix = int(line.split()[-1])
            hp_dict[pix].append(line)
    # to here can be a function that
    # accepts infilename and returns
    # hp_dict

    pixel_results = {}
    #for i in range(hp.nside2npix(nside)):
    for i in pixels:
        print(i)
        # Probably don't need to repeat the vector neighbor calculation.
        # This can just be stored.
        vec = hp.pix2vec(nside, i, nest=True)
        neighbors = hp.query_disc(nside, vec, angDeg*np.pi/180., inclusive=True, nest=True)
        lines = []
        for pix in neighbors:
            for line in hp_dict[pix]:
                lines.append(line)
        if len(lines) > 0:
            pixel_results[i] = cluster_func(t_ref, g_gdot_pairs, vec, lines)
            
    return pixel_results
        



We need to do some training to determine what is a reasonable resolution for z and z-dot (or gamma and gamma-dot).   We also need to train for dt, the scale factor to relate the angular rates to the angles.  I'll first generate a very fine grid.

In [ ]:
def member_counts(k, sep='|', suff='_'):
    keys = k.split(sep)
    stems = [key.split(suff)[0] for key in keys]
    stem_counter = Counter(stems)
    return stem_counter

def unique_clusters(test_set):
    check_dict = {}
    errs=[]
    for k in test_set:
        stem_counter = member_counts(k)
        if len(stem_counter)>1:
            errs.append(k)
        else:
            for k, v in stem_counter.items():
                if k not in check_dict:
                    check_dict[k] = v
                elif v > check_dict[k]:
                    check_dict[k] = v
    return check_dict, errs


Not only do we want to tune over the velocity scaling factor dt and the cluster size d, we also want to get a sense for how finely we need to sample the z, zdot space to have a good detection efficiency.  I am guessing that with a bigger dt and a bigger d we can get away with a courser grid of z, zdot, but a larger d comes with a somewhat larger contamination rate from overlapping clusters.

In [ ]:
zs = np.arange(2.5, 2.6, 0.5)
#zs = np.arange(3.33, 3.5, 0.5)
#zs = np.arange(10.0, 10.1, 0.5)
zdots = np.arange(-1e-2, 1.1e-2, 2.0e-3)
z_zdots = [(x,y) for x in zs for y in zdots]

gs = [0.3, 0.35, 0.4, 0.45, 0.5]
gs =[0.4]
gdots = [-0.004, -0.003, -0.002, -0.001, 0.0, 0.001, 0.002, 0.003, 0.004]
g_gdots = [(x,y) for x in gs for y in gdots]

# This could accept a cluster_sky regions function, so that there is only one version of this
# code.

# And it should accept dt and rad ranges.

def do_training_run(pixels, infilename, t_ref, 
                    cluster_sky_function=cluster_sky_regions,
                    g_gdots=g_gdots, mincount=3, 
                    dts=np.arange(5, 30, 5), 
                    radii=np.arange(0.0001, 0.0100, 0.0001)):
    
    master = cluster_sky_function(g_gdots, pixels, t_ref, infilename)

    results_dict = {}

    rates_dict={}
    for dt in dts:
        for rad in radii:
            cluster_counter = Counter()
            for pix, d in master.items():
                for z_zdot, arrows in d.items():

                    # The bit from here
                    i = 0
                    label_dict={}
                    combined=[]
                    for k, cx, mx, cy, my, t in arrows:
                        label_dict[i] = k
                        combined.append([cx, mx*dt, cy, my*dt])
                        i +=1
                    points=np.array(combined)
                    # to here can be a function,
                    # that takes arrows and dt and
                    # returns label_dict and points array
                    
                    # The bit from here
                    tree = scipy.spatial.cKDTree(points)
                    matches = tree.query_ball_tree(tree, rad)
                    # to here can be a function, that takes
                    # points are rad and returns tree and
                    # matches
                    
                    for j, match in enumerate(matches):
                        if len(match)>=mincount:
                            cluster_list =[]
                            tracklet_params=[]
                            for idx in match:
                                cluster_list.append(label_dict[idx].strip())
                                #tracklet_params.append(combined[idx])
                            cluster_key='|'.join(sorted(cluster_list))
                            cluster_counter.update({cluster_key: 1})
            
            # This region from here 
            errs = 0
            for i, k in enumerate(cluster_counter.keys()):
                keys = k.split('|')
                stems = [key.split('_')[0] for key in keys]
                stem_counter = Counter(stems)
                if len(stem_counter)>1:
                    errs +=1
            # to here can be a function that takes a concatenated
            # cluster ID and returns the number of errors

            rates_dict[dt, rad] = cluster_counter.keys(), errs

    for dt in dts:
        values = []
        for k, v in rates_dict.items():
            dtp, d = k
            if dtp==dt:
                test_set = list(v[0])    
                ncs, nes = len(unique_clusters(test_set)[0]), len(unique_clusters(test_set)[1])
                values.append((d, ncs, nes, test_set))

        values = sorted(values, key=lambda v: v[0])
        ds = [v[0] for v in values]
        nclusters = [v[1] for v in values]
        nerrors = [v[2] for v in values]
        #keys = [v[3] for v in values]
        results_dict[dt] = ds, nclusters, nerrors
     
    return results_dict

Now I am going to set up five training runs for the gamma=0.4 distance class.  I will use a very fine grid in gamma-dot (1.25e-3/2.5 = 5e-4.

We are looking for the parameters that give us the highest completeness for essentially no errors.  I will look for the parameters, among the five training runs, that achieve that.  Then I will test it on other time slices and data sets.

Then I will make the grid spacing more coarse, to see if it is possible to achieve the same results, or nearly the same, for less computational cost.

Later, I need to look at different distance classes, perhaps gamma=0.2, 0.3, 0.4, 0.5, 0.6

In [ ]:
nside=8

#gs = [0.3, 0.35, 0.4, 0.45, 0.5]
gs = [0.4]
#gdots = [-0.004, -0.003, -0.002, -0.001, 0.0, 0.001, 0.002, 0.003, 0.004]
gdots = [-0.004, -0.002, 0.0, 0.002, 0.004]
g_gdots = [(x,y) for x in gs for y in gdots]

pix_runs = {}
nside=8

# Looping over five lunation centers, separated by 3 months each
for n in [-11, -14, -17, -20, -23]:
    infilename='data/UnnObs_Training_1_line_A_%.1lf_pm15.0_r2.5.trans' % (lunation_center(n))
    pickle_filename = infilename.rstrip('trans') + '.pickle'
    for pix in range(hp.nside2npix(nside)):
        pix_runs[pix] = do_training_run([pix], infilename, lunation_center(n), g_gdots=g_gdots)
    with open(pickle_filename, 'wb') as handle:
        pickle.dump(pix_runs, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [ ]:
def output_sky_regions(pixels, infilename='data/itf_new_1_line_2457397.5_pm45.0_r2.5.trans', nside=8, n=-11, angDeg=7.5):
    hp_dict = defaultdict(list)
    with open(infilename) as file:
        i=0
        for line in file:
            if line.startswith('#'):
                continue
            pix = int(line.split()[-1])
            hp_dict[pix].append(line)
            i += 1
            
    pixel_results = {}
    for i in pixels:
        # Probably don't need to repeat the vector neighbor calculation.
        # This can just be stored.
        vec = hp.pix2vec(nside, i, nest=True)
        neighbors = hp.query_disc(nside, vec, angDeg*np.pi/180., inclusive=True, nest=True)
        lines = []
        for pix in neighbors:
            for line in hp_dict[pix]:
                lines.append(line)

    return lines
        

In [ ]:
def accessible_clusters(pixels, infilename='data/UnnObs_Training_1_line_A_2457397.5_pm15.0_r2.5.trans', mincount=3):
    true_counts={}
    mergedCounter_dict = {}
    mergedTime_dict = {}
    for pix in pixels:
        lines = output_sky_regions([pix], infilename=infilename)
        #print(pix, len(lines))
        trackletCounter = Counter()
        tracklet_time = defaultdict(float)
        for line in lines:
            trackletID=line.split()[0]
            trackletCounter.update({trackletID : 1})
            jd_tdb = float(line[43:57])
            tracklet_time[trackletID] = jd_tdb

        mergedCounter = Counter()
        time_dict = defaultdict(list)
        for k, v in trackletCounter.items():
            mergedCounter.update({k[:-4]:1})
            time_dict[k[:-4]].append(tracklet_time[k])
        true_counts[pix]=len([k for k, v in mergedCounter.items() if v>=mincount])
        mergedCounter_dict[pix]=mergedCounter
        mergedTime_dict[pix]=time_dict
    
    return true_counts, mergedCounter_dict, mergedTime_dict

In [ ]:
def make_histogram(mergedCounter):
    hist_dict=defaultdict(list)
    for k, v in mergedCounter.items():
        hist_dict[v].append(k)
    return hist_dict
        

In [ ]:
for n in [-11]:
    infilename='data/UnnObs_Training_1_line_A_%.1lf_pm15.0_r2.5.trans' % (lunation_center(n))
    pickle_filename = infilename.rstrip('trans') + 'pickle'
    with open(pickle_filename, 'rb') as handle:
        pix_runs = pickle.load(handle)

In [ ]:
for n in [-11]:
    infilename='data/UnnObs_Training_1_line_A_%.1lf_pm15.0_r2.5.trans' % (lunation_center(n))
    pickle_filename = infilename.rstrip('trans') + 'trn2_pickle'
    with open(pickle_filename, 'rb') as handle:
        pix_runs = pickle.load(handle)

In [ ]:
# Should save these results in files
true_count_dict, mergedCounter_dict, mergedTime_dict=accessible_clusters(list(pix_runs.keys()), infilename=infilename)
true_count=sum(true_count_dict.values())
true_count

Do something general with the snippet below.

In [ ]:
for k, v in mergedTime_dict[100].items():
    if len(v)>2:
        print("%s %4d %6.2lf" % (k, len(v), max(v)-min(v)))

In [ ]:
num_occ=Counter()
for pix in range(hp.nside2npix(nside)):
    hist=make_histogram(mergedCounter_dict[pix])
    for k, v in hist.items():
        num_occ.update({k: len(v)})
        
num_occ

In [ ]:

for dt in np.arange(10, 80, 10):
    pixels=list(pix_runs.keys())
    ds = pix_runs[pixels[0]][dt][0]
    nclusters = pix_runs[pixels[0]][dt][1]
    nerrors = pix_runs[pixels[0]][dt][2]
    for pix in pixels[1:]:
        nclusters = list(map(add, nclusters, pix_runs[pix][dt][1]))
        nerrors = list(map(add, nerrors, pix_runs[pix][dt][2]))
    nclusters=np.array(nclusters)
    
    plt.plot(ds, nclusters, label=dt)

plt.xscale("log", nonposx='clip')
plt.axhline(true_count, ls='dashed')
plt.xlabel('d (cluster radius)')
plt.ylabel('N clusters')
plt.text(0.007, 400, r'$\gamma=0.4$', fontsize=15)
plt.legend()
plt.savefig('nclusters_vs_d_z2.5.pdf')
plt.show()
# 

In [ ]:
for dt in np.arange(1, 16, 1):
    pixels=list(pix_runs.keys())
    ds = pix_runs[pixels[0]][dt][0]
    nclusters = pix_runs[pixels[0]][dt][1]
    nerrors = pix_runs[pixels[0]][dt][2]
    for pix in pixels[1:]:
        nclusters = list(map(add, nclusters, pix_runs[pix][dt][1]))
        nerrors = list(map(add, nerrors, pix_runs[pix][dt][2]))
    nclusters=np.array(nclusters)
    
    plt.plot(ds, nclusters, label=dt)

plt.axhline(true_count, ls='dashed')
plt.xlabel('d (cluster radius)')
plt.ylabel('N clusters')
#plt.text(0.005, 400, r'$\gamma=0.4$', fontsize=15)
#plt.legend()
plt.savefig('nclusters_vs_d_z2.5.pdf')
plt.show()
# 

In [ ]:
for dt in np.arange(10, 80, 10):
    pixels=list(pix_runs.keys())
    ds = pix_runs[pixels[0]][dt][0]
    nclusters = pix_runs[pixels[0]][dt][1]
    nerrors = pix_runs[pixels[0]][dt][2]
    for pix in pixels[1:]:
        nclusters = list(map(add, nclusters, pix_runs[pix][dt][1]))
        nerrors = list(map(add, nerrors, pix_runs[pix][dt][2]))
    nclusters=np.array(nclusters)
    nerrors=np.array(nerrors)
    '''
    if dt==50:
        for d, nc, ne in zip(ds, nclusters, nerrors):
            print(d, nc, ne)
    '''
            
    
    plt.plot(ds, nerrors, label=dt)

plt.xscale("log", nonposx='clip')
plt.ylim((0,3000))
plt.xlabel('d (cluster radius)')
plt.ylabel('N errors')
plt.text(0.0005, 1000, r'$\gamma=0.4$', fontsize=15)
plt.legend()
plt.savefig('nerrors_vs_d_z2.5.pdf')
plt.show()
# 



In [ ]:
for dt in np.arange(1, 16, 1):
    pixels=list(pix_runs.keys())
    ds = pix_runs[pixels[0]][dt][0]
    nclusters = pix_runs[pixels[0]][dt][1]
    nerrors = pix_runs[pixels[0]][dt][2]
    for pix in pixels[1:]:
        nclusters = list(map(add, nclusters, pix_runs[pix][dt][1]))
        nerrors = list(map(add, nerrors, pix_runs[pix][dt][2]))
    nclusters=np.array(nclusters)
    nerrors=np.array(nerrors)
    '''
    if dt==50:
        for d, nc, ne in zip(ds, nclusters, nerrors):
            print(d, nc, ne)
    '''
            
    
    plt.plot(ds, nerrors, label=dt)

plt.ylim((0,200))
plt.xlabel('d (cluster radius)')
plt.ylabel('N errors')
#plt.text(0.0005, 1000, r'$\gamma=0.4$', fontsize=15)
#plt.legend()
plt.savefig('nerrors_vs_d_z2.5.pdf')
plt.show()
# 




In [ ]:

for dt in np.arange(10, 80, 10):
    pixels=list(pix_runs.keys())
    ds = pix_runs[pixels[0]][dt][0]
    nclusters = pix_runs[pixels[0]][dt][1]
    nerrors = pix_runs[pixels[0]][dt][2]
    for pix in pixels[1:]:
        nclusters = list(map(add, nclusters, pix_runs[pix][dt][1]))
        nerrors = list(map(add, nerrors, pix_runs[pix][dt][2]))
    nclusters=np.array(nclusters)
    nerrors=np.array(nerrors)    
    
    plt.plot(nerrors/true_count, nclusters/true_count, label=dt)

plt.xlim((0.0001, 0.5))
plt.xscale("log", nonposx='clip')
plt.ylim((0, 1))
plt.xlabel('Error rate')
plt.ylabel('Fraction complete')
plt.text(0.05, 0.2, r'$\gamma=0.4$', fontsize=15)
plt.legend()
plt.savefig('AUCish_z2.5.pdf')
plt.show()
# 




In [ ]:
for dt in np.arange(1, 21, 1):
    pixels=list(pix_runs.keys())
    ds = pix_runs[pixels[0]][dt][0]
    nclusters = pix_runs[pixels[0]][dt][1]
    nerrors = pix_runs[pixels[0]][dt][2]
    for pix in pixels[1:]:
        nclusters = list(map(add, nclusters, pix_runs[pix][dt][1]))
        nerrors = list(map(add, nerrors, pix_runs[pix][dt][2]))
    nclusters=np.array(nclusters)
    nerrors=np.array(nerrors)    
    
    plt.plot(nerrors/true_count, nclusters/true_count, label=dt)

plt.xlim((0, 0.02))
plt.ylim((0, 1))
plt.xlabel('Error rate')
plt.ylabel('Fraction complete')
plt.text(0.05, 0.2, r'$\gamma=0.4$', fontsize=15)
#plt.legend()
plt.savefig('AUCish_z2.5.pdf')
plt.show()
# 

I want to plot where the errors are occurring, to see if they are happening near quadrature.  It doesn't seem to be the case.

Here's a working goal.  Let's try to achieve 90% completeness with minimal errors.  From the runs I have done, dt=20 does achieve 90% completeness, with a 4% error rate.  

I can try a few more dt values, around 20.

An alternative approach is to aim to be less complete but with a lower error rate.  And then remove those clusters and iterate.  

The errors are false positives.  That is, we say they are clusters, but they are at least contaminated.

From the plot below it looks like we want dt=10 days, a completeness of 85%, and an error rate of 1%.  We achieve that at d=0.0008.

Actually, if we go with dt=50 days, we can achieve 50% completeness with not recorded errors (or an error rate of less than 0.1%.  Why don't we do with that for a first passs?



In [ ]:
n=-23
dt=10
earth_vec = Observatories.getObservatoryPosition('500', lunation_center(n))
# Actually, this should be related to the angle between the direction of
# Earth's motion and the direction of the observations.
errs=0
clusts=0
trues=0
for pix in list(pix_runs.keys()):
    nclusters = pix_runs[pixels[pix]][dt][1][4]
    nerrors = pix_runs[pixels[pix]][dt][2][4]

    ntrue = true_count_dict[pix]
    vec = hp.pix2vec(nside, pix, nest=True)
    ang=180./np.pi * np.arccos(np.dot(earth_vec/np.linalg.norm(vec), vec))
    if ang<180:
        #print("%5d %5d %5d %5d %7.2lf" % (pix, nclusters, nerrors, ntrue, ang))
        errs += nerrors
        clusts += nclusters
        trues += ntrue
errs, clusts, trues

In [ ]:
pix_runs

These are combinations that resulted in now errors.
-11, dt=10, d=[5]
-11, dt=20, d=[8]
-12, dt=10, d=[11]
-12, dt=20, d=[17]
-13, dt=10, d=[20]
-13, dt=20, d=[26]
-15, dt=10, d=[6]
-15, dt=20, d=[10] not quite 50%

Here I am timing a run

In [ ]:
gs = [0.4]
gdots = [-0.004, -0.003, -0.002, -0.001, 0.0, 0.001, 0.002, 0.003, 0.004]
g_gdots = [(x,y) for x in gs for y in gdots]

pix_runs = {}
nside=8

pixels= range(hp.nside2npix(nside))

# Looping over five lunation centers, separated by 3 months each
for n in [-11]:
    infilename='data/UnnObs_Training_1_line_A_%.1lf_pm15.0_r2.5.trans' % (lunation_center(n))
    pickle_filename = infilename.rstrip('trans') + 'trn2_pickle'
    for pix in range(hp.nside2npix(nside)):
        pix_runs[pix] = do_training_run([pix], infilename, lunation_center(n), g_gdots=g_gdots)
    with open(pickle_filename, 'wb') as handle:
        pickle.dump(pix_runs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
gs = [0.4]
gdots = [-0.004, -0.003, -0.002, -0.001, 0.0, 0.001, 0.002, 0.003, 0.004]
g_gdots = [(x,y) for x in gs for y in gdots]

nside=8
n=-11

pixels= range(hp.nside2npix(nside))

def do_run(g_gdots=g_gdots, pixels=pixels, infilename='data/itf_new_1_line_2457397.5_pm15.0_r2.5.trans', nside=nside, n=n, angDeg=7.5, dt=5, rad=0.0005, mincount=3):

    print('tranforming')
    t_ref = lunation_center(n)
    master=cluster_sky_regions(g_gdots, pixels, t_ref, infilename, nside=nside, angDeg=angDeg)

    print('clustering')
    #results_dict = {}
    # In the process of clustering here, we need to
    # keep track of the 'best' or biggest cluster that
    # each tracklet is a member of.
    #rates_dict={}
    cluster_counter = Counter()
    for pix, d in master.items():
        for g_gdot, arrows in d.items():
            i = 0
            label_dict={}
            combined=[]
            for k, cx, mx, cy, my, t in arrows:
                label_dict[i] = k
                combined.append([cx, mx*dt, cy, my*dt])
                i +=1
            points=np.array(combined)
            tree = scipy.spatial.cKDTree(points)
            matches = tree.query_ball_tree(tree, rad)
            for j, match in enumerate(matches):
                if len(match)>mincount:
                    cluster_list =[]
                    tracklet_params=[]
                    for idx in match:
                        cluster_list.append(label_dict[idx].strip())
                        tracklet_params.append(combined[idx])
                    cluster_key='|'.join(sorted(cluster_list))
                    cluster_counter.update({cluster_key: 1})
                    
    return cluster_counter


In [ ]:
%time first_real_run = do_run(infilename='data/UnnObs_Training_1_line_A_2457308.5_pm15.0_r2.5.trans', n=-14, dt=3)

In [ ]:
lunation_center(-14)

In [ ]:
len(first_real_run)

In [ ]:
with open('itf_new_1_line_2457397.5_pm15.0_z2.5.pickle', 'wb') as handle:
    pickle.dump(first_real_run, handle, protocol=pickle.HIGHEST_PROTOCOL)

    
with open('itf_new_1_line_2457397.5_pm15.0_z2.5.pickle', 'rb') as handle:
    first_real_run_b = pickle.load(handle)

print(first_real_run == first_real_run_b)

In [ ]:
i = 0
for k, v in first_real_run[0].items():
    print(k, v)
    i += 1
    if i>5:
        break

In [ ]:
first_keys = list(first_real_run.keys())
first_keys = sorted(first_keys, key=lambda v: len(v.split('|')), reverse=True)

In [ ]:
def get_original_tracklets_dict(filename='data/UnnObs_1_line.txt'):
    tracklets = defaultdict(list)
    with open(filename) as infile:
        for line in infile:
            if not line.startswith('#'):
                desig = line[0:12].strip()
                tracklets[desig].append(line)
    return tracklets

In [ ]:
original_tracklets_dict = get_original_tracklets_dict()

In [ ]:
UnnObs_tracklets_dict = get_original_tracklets_dict(filename='data/UnnObs_1_line.txt')

Want to make a function that generates an MPC-format linkage file to be emailed to Gareth.

In [ ]:
def get_original_data(cluster_key, tracklets_dict):
    lines=[]
    trackletIDs = cluster_key.split('|')
    for trackletID in trackletIDs:
        for line in tracklets_dict[trackletID]:
            lines.append(line)
    lines = sorted(lines, key=lambda x: x[15:31])
    return lines

In [ ]:
with open('obs.obs', 'w') as file:
    for cluster_key in first_keys:
        file.write("%s\n" %(cluster_key))
        lines=get_original_data(cluster_key, original_tracklets_dict)
        for line in lines:
            file.write(line)

In [ ]:
for key in first_keys:
    lines = get_original_data(key[:-4], UnnObs_tracklets_dict)
    print(key, lines)

In [ ]:
# Should just passing the selection function, to avoid code duplication
def generate_sky_region_files(infilename='data/itf_new_1_line_2457397.5_pm15.0_r2.5.trans', nside=8, n=-11, angDeg=5.5, g=0.4, gdot=0.0):
    hp_dict = defaultdict(list)
    with open(infilename) as file:
        for line in file:
            if line.startswith('#'):
                continue
            pix = int(line.split()[-1])
            hp_dict[pix].append(line)

    for i in range(hp.nside2npix(nside)):
        vec = hp.pix2vec(nside, i, nest=True)
        neighbors = hp.query_disc(32, vec, angDeg*np.pi/180., inclusive=True, nest=True)
        lines = []
        for pix in neighbors:
            for line in hp_dict[pix]:
                lines.append(line)
        outfilename = infilename.rstrip('.trans') + '_hp_' + ('%03d' % (i)) + '_g'+ ('%.2lf' % (g))+'_gdot' + ('%+5.1le' % (gdot))
        if len(lines) > 0:
            select_positions_z(lunation_center(n), g, gdot, vec, lines, outfilename)





In [ ]:
lunation_center(-15)

In [ ]:
generate_sky_region_files(infilename='data/UnnObs_Training_1_line_A_2457279.5_pm15.0_r2.5.trans', n=-15)

In [ ]:
def generate_sky_region_files_v2(infilename='data/itf_new_1_line_2457397.5_pm15.0_r2.5.trans', nside=8, n=-11, angDeg=5.5, g=0.4, gdot=0.0):
    hp_dict = defaultdict(list)
    with open(infilename) as file:
        for line in file:
            if line.startswith('#'):
                continue
            pix = int(line.split()[-1])
            hp_dict[pix].append(line)

    for i in range(hp.nside2npix(nside)):
        vec = hp.pix2vec(nside, i, nest=True)
        neighbors = hp.query_disc(32, vec, angDeg*np.pi/180., inclusive=True, nest=True)
        lines = []
        for pix in neighbors:
            for line in hp_dict[pix]:
                lines.append(line)
        outfilename = infilename.rstrip('.trans') + '_hp_' + ('%03d' % (i)) + '_z'+ ('%.2lf' % (z0))+'_zdot' + ('%+5.1le_v2' % (zdot0))
        if len(lines) > 0:
            select_positions_z_v2(lunation_center(n), g, gdot, vec, lines, outfilename)






In [ ]:
z0=3.0
for n in range(-11, -9):
    infilename = 'itf_new_1_line_'+str(lunation_center(n))+'_pm15.0_r2.5.trans'
    print(infilename)
    generate_sky_region_files(infilename=infilename, n=n, z0=z0)

        

In [ ]:
import matplotlib.cm as cm
from matplotlib.colors import Normalize

def make_figure(filename):
    plt.ioff()
    mxs, cxs, mys, cys, dts =[], [], [], [], []
    for line in open(filename):
        if line.startswith('#'):
            continue
        desig, cx, mx, cy, my, dt = line.split()
        mxs.append(float(mx))
        cxs.append(float(cx))
        mys.append(float(my))
        cys.append(float(cy))
        dts.append(float(dt))
    
    fig=plt.figure(figsize=(18, 16))

    #norm = Normalize()
    #norm.autoscale(colors)
    
    colormap = cm.inferno

    plt.quiver(cxs, cys, mxs, mys, dts, scale=0.3, width=0.0003)

    plt.xlim(-0.2, 0.2)
    plt.ylim(-0.2, 0.2)
    plt.xlabel('alpha')
    plt.ylabel('beta')
    outfile = filename+'.pdf'
    plt.savefig(outfile)
    plt.close()
    plt.ion()


In [ ]:
def make_figure_zoom(filename):
    plt.ioff()
    mxs, cxs, mys, cys, dts =[], [], [], [], []
    for line in open(filename):
        if line.startswith('#'):
            continue
        desig, cx, mx, cy, my, dt = line.split()
        mxs.append(float(mx))
        cxs.append(float(cx))
        mys.append(float(my))
        cys.append(float(cy))
        dts.append(float(dt))
    
    fig=plt.figure(figsize=(18, 16))

    #norm = Normalize()
    #norm.autoscale(colors)
    
    colormap = cm.inferno

    plt.quiver(cxs, cys, mxs, mys, dts, scale=0.3, width=0.0003)

    plt.xlim(-0.1, 0.1)
    plt.ylim(-0.1, 0.1)
    plt.xlabel('alpha')
    plt.ylabel('beta')
    outfile = filename+'_zm.pdf'
    plt.savefig(outfile)
    plt.close()
    plt.ion()

In [ ]:
lunation_center(-11)

In [ ]:
zdot0=+2.5e-3
for z0 in [2.0, 2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 2.9, 3.0, 3.1, 3.2, 3.3, 3.4, 3.5]:
    for n in range(-11, -10):
        infilename = 'data/UnnObs_Training_1_line_A_'+str(lunation_center(n))+'_pm45.0_r2.5.trans'
        print(infilename)
        generate_sky_region_files_v2(infilename=infilename, n=n, z0=z0, zdot0=zdot0)



In [ ]:
n=-15
gs = [0.4]
gdots = [-0.004, -0.003, -0.002, -0.001, 0.0, 0.001, 0.002, 0.003, 0.004]
for gdot in gdots:
    infilename = 'data/UnnObs_Training_1_line_A_'+str(lunation_center(n))+'_pm45.0_r2.5.trans'
    print(infilename)
    generate_sky_region_files(infilename=infilename, n=n, g=g, gdot=gdot)


In [ ]:
z0=3.1
for zdot0 in [-1.0e-2, -8.0e-3, -6.0e-3, -4.0e-3, -2.0e-3, 0.0e-3, 2.0e-3, 4.0e-3, 6.0e-3, 8.0e-3, 1.0e-2]:
    for n in range(-11, -10):
        infilename = 'data/UnnObs_Training_1_line_A_'+str(lunation_center(n))+'_pm45.0_r2.5.trans'
        print(infilename)
        generate_sky_region_files_v2(infilename=infilename, n=n, z0=z0, zdot0=zdot0)



In [ ]:
z0=3.1
for zdot0 in [-1.0e-2, -8.0e-3, -6.0e-3, -4.0e-3, -2.0e-3, 0.0e-3, 2.0e-3, 4.0e-3, 6.0e-3, 8.0e-3, 1.0e-2]:
    for n in range(-11, -10):
        infilename = 'data/UnnObs_Training_1_line_A_'+str(lunation_center(n))+'_pm45.0_r2.5.trans'
        filename = infilename.rstrip('.trans') + '_hp_378' + ('_z%4.2lf_zdot%+3.1le_v2' % (z0, zdot0))
        make_figure(filename)
        make_figure_zoom(filename)
        

In [ ]:
gs = [0.4]
gdots = [-0.004, -0.003, -0.002, -0.001, 0.0, 0.001, 0.002, 0.003, 0.004]
g_gdots = [(x,y) for x in gs for y in gdots]

n=-15
for g_gdot in g_gdots:
    g, gdot = g_gdot
    infilename = 'data/UnnObs_Training_1_line_A_'+str(lunation_center(n))+'_pm45.0_r2.5.trans'
    filename = infilename.rstrip('.trans') + '_hp_000' + ('_g%4.2lf_gdot%+3.1le' % (g, gdot))
    make_figure(filename)
    


In [ ]:
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.00_zdot+2.5e-03_v2')
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.10_zdot+2.5e-03_v2')
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.20_zdot+2.5e-03_v2')
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.30_zdot+2.5e-03_v2')
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.40_zdot+2.5e-03_v2')
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.50_zdot+2.5e-03_v2')
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.60_zdot+2.5e-03_v2')
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.70_zdot+2.5e-03_v2')
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.80_zdot+2.5e-03_v2')
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.90_zdot+2.5e-03_v2')
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z3.00_zdot+2.5e-03_v2')
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z3.10_zdot+2.5e-03_v2')
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z3.20_zdot+2.5e-03_v2')
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z3.30_zdot+2.5e-03_v2')
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z3.40_zdot+2.5e-03_v2')
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z3.50_zdot+2.5e-03_v2')

In [ ]:
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.00_zdot+2.5e-03_v2')
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.10_zdot+2.5e-03_v2')
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.20_zdot+2.5e-03_v2')
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.30_zdot+2.5e-03_v2')
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.40_zdot+2.5e-03_v2')
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.50_zdot+2.5e-03_v2')
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.60_zdot+2.5e-03_v2')
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.70_zdot+2.5e-03_v2')
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.80_zdot+2.5e-03_v2')
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.90_zdot+2.5e-03_v2')
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z3.00_zdot+2.5e-03_v2')
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z3.10_zdot+2.5e-03_v2')
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z3.20_zdot+2.5e-03_v2')
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z3.30_zdot+2.5e-03_v2')
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z3.40_zdot+2.5e-03_v2')
make_figure('data/UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z3.50_zdot+2.5e-03_v2')

In [ ]:
make_figure('UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.50_zdot+0.0e+00')
make_figure('UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.50_zdot+5.0e-03')
make_figure('UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.50_zdot-5.0e-03')
make_figure('UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.50_zdot+2.5e-03')
make_figure('UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.50_zdot-2.5e-03')
make_figure('UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.50_zdot+7.5e-03')
make_figure('UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.50_zdot-7.5e-03')

In [ ]:
make_figure('UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z3.00_zdot+0.0e+00')
make_figure('UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z3.00_zdot+5.0e-03')
make_figure('UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z3.00_zdot-5.0e-03')
make_figure('UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z3.00_zdot+2.5e-03')
make_figure('UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z3.00_zdot-2.5e-03')
make_figure('UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z3.00_zdot+7.5e-03')
make_figure('UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z3.00_zdot-7.5e-03')

In [ ]:
make_figure('UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.00_zdot+0.0e+00')
make_figure('UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.00_zdot+5.0e-03')
make_figure('UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.00_zdot-5.0e-03')
make_figure('UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.00_zdot+2.5e-03')
make_figure('UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.00_zdot-2.5e-03')
make_figure('UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.00_zdot+7.5e-03')
make_figure('UnnObs_Training_1_line_A_2457397.5_pm45.0_r2.5_hp_378_z2.00_zdot-7.5e-03')

Here's a game plan:

* Verify the dark time numbers --DONE.
* Stick with a smallish time window (+/- 15 days) --DONE.
* Do heliocentric transformation for r=2.5 AU (rdot=0) --DONE.
* Do selection_positions_z for all the time slices.  This will generate a large number of files.
* Identify the largest files.  Find a way to not generate empty files.
* Run clustering on each file, identifying candidate clusters.
* Somehow vet the candidate clusters.  I can think of three ways: (1) use Gareth's code, (2) use Bernstein's code, or (3) fit a simple linear model.   The third would be a very quick way of clustering tracklets within a lunation.

The r2.5 run is just to separate the tracklets into sky regions.  Given those sky regions, I can examine a runs of z and zdot.  It seems that a simple grid of a few values in z (3 or 4) and zdot (7 or 8) might be enough.

Perhaps it is possible to do all the clustering for a sky region of a given time slice all in one pass.  It's not necessary to write out the intermediate files.  Given a set of tracklets, transform for a (z, zdot) value and then search for clusters.

If a cluster is found, make a key of the tuple of sorted trackletIDs and then add that key to a dictionary or counter. 

Here's a new game plan:

* Select 5 different disjoint time slices.  These will be +/- 15 days and +/- 45 days.
* Run the training on those time slices for all sky regions for both +/- 15 days and +/- 45 days.  The former requires only the gamma-dot parameter to be varied, for a few values of gammma.  The latter probably requires a finer grid in gamma and gamma-dot.  In both cases, the hyperparameters that we are tuning are the cluster radius and the time factor dt.
* If I want to do repeated universal kepler steps, I need to make that fast in python.